In [10]:
import requests
from psycopg2 import connect, sql

In [11]:
conn_string = "dbname='etl_bites' user='' password='' host='localhost' port='5432'"

In [12]:
# Extract
def get_data_from_api(url):
    response = requests.get(url)
    return response.json()

todos_url = 'https://jsonplaceholder.typicode.com/todos'
users_url = 'https://jsonplaceholder.typicode.com/users' 

In [18]:
todos_data = get_data_from_api(todos_url)
todos_data

[{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False},
 {'userId': 1,
  'id': 2,
  'title': 'quis ut nam facilis et officia qui',
  'completed': False},
 {'userId': 1, 'id': 3, 'title': 'fugiat veniam minus', 'completed': False},
 {'userId': 1, 'id': 4, 'title': 'et porro tempora', 'completed': True},
 {'userId': 1,
  'id': 5,
  'title': 'laboriosam mollitia et enim quasi adipisci quia provident illum',
  'completed': False},
 {'userId': 1,
  'id': 6,
  'title': 'qui ullam ratione quibusdam voluptatem quia omnis',
  'completed': False},
 {'userId': 1,
  'id': 7,
  'title': 'illo expedita consequatur quia in',
  'completed': False},
 {'userId': 1,
  'id': 8,
  'title': 'quo adipisci enim quam ut ab',
  'completed': True},
 {'userId': 1,
  'id': 9,
  'title': 'molestiae perspiciatis ipsa',
  'completed': False},
 {'userId': 1,
  'id': 10,
  'title': 'illo est ratione doloremque quia maiores aut',
  'completed': True},
 {'userId': 1,
  'id': 11,
  'title': 'vero rerum

In [15]:
users_url = "https://jsonplaceholder.typicode.com/users"

users_data = get_data_from_api(users_url)
users_data

[{'id': 1,
  'name': 'Leanne Graham',
  'username': 'Bret',
  'email': 'Sincere@april.biz',
  'address': {'street': 'Kulas Light',
   'suite': 'Apt. 556',
   'city': 'Gwenborough',
   'zipcode': '92998-3874',
   'geo': {'lat': '-37.3159', 'lng': '81.1496'}},
  'phone': '1-770-736-8031 x56442',
  'website': 'hildegard.org',
  'company': {'name': 'Romaguera-Crona',
   'catchPhrase': 'Multi-layered client-server neural-net',
   'bs': 'harness real-time e-markets'}},
 {'id': 2,
  'name': 'Ervin Howell',
  'username': 'Antonette',
  'email': 'Shanna@melissa.tv',
  'address': {'street': 'Victor Plains',
   'suite': 'Suite 879',
   'city': 'Wisokyburgh',
   'zipcode': '90566-7771',
   'geo': {'lat': '-43.9509', 'lng': '-34.4618'}},
  'phone': '010-692-6593 x09125',
  'website': 'anastasia.net',
  'company': {'name': 'Deckow-Crist',
   'catchPhrase': 'Proactive didactic contingency',
   'bs': 'synergize scalable supply-chains'}},
 {'id': 3,
  'name': 'Clementine Bauch',
  'username': 'Samantha

In [22]:
# Transform
def transform_todos(todos, users):
    user_dict = {user['id']: user for user in users}
    completed_tasks_count = {user['id']: 0 for user in users}

    for todo in todos:
        if todo['completed']:
            completed_tasks_count[todo['userId']] += 1

   
    result = []
    for user_id, count in completed_tasks_count.items():
        result.append({
            'user_id': user_id,
            'name': user_dict[user_id]['name'],
            'completed_tasks': count
        })
    
    return result

transformed_data = transform_todos(todos_data, users_data)
transformed_data


[{'user_id': 1, 'name': 'Leanne Graham', 'completed_tasks': 11}, {'user_id': 2, 'name': 'Ervin Howell', 'completed_tasks': 8}, {'user_id': 3, 'name': 'Clementine Bauch', 'completed_tasks': 7}, {'user_id': 4, 'name': 'Patricia Lebsack', 'completed_tasks': 6}, {'user_id': 5, 'name': 'Chelsey Dietrich', 'completed_tasks': 12}, {'user_id': 6, 'name': 'Mrs. Dennis Schulist', 'completed_tasks': 6}, {'user_id': 7, 'name': 'Kurtis Weissnat', 'completed_tasks': 9}, {'user_id': 8, 'name': 'Nicholas Runolfsdottir V', 'completed_tasks': 11}, {'user_id': 9, 'name': 'Glenna Reichert', 'completed_tasks': 8}, {'user_id': 10, 'name': 'Clementina DuBuque', 'completed_tasks': 12}]


In [25]:
def execute_query_postgresql(conn_string, query):
    with connect(conn_string) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            conn.commit()

create_api_data_table = '''
CREATE TABLE IF NOT EXISTS transformed_data (
    user_id INTEGER NOT NULL,
    name TEXT NOT NULL,
    completed_tasks TEXT NOT NULL
);
'''

execute_query_postgresql(conn_string, create_api_data_table)

def insert_data_to_postgresql(conn_string, table_name, data):
    with connect(conn_string) as conn:
        with conn.cursor() as cur:
            for item in data:
                query = sql.SQL("INSERT INTO {} (user_id, name, completed_tasks) VALUES (%s, %s, %s)").format(sql.Identifier(table_name))
                cur.execute(query, (item['user_id'], item['name'], item['completed_tasks']))
        conn.commit()

table_name = "transformed_data"
insert_data_to_postgresql(conn_string, table_name, transformed_data)
transformed_data

[{'user_id': 1, 'name': 'Leanne Graham', 'completed_tasks': 11},
 {'user_id': 2, 'name': 'Ervin Howell', 'completed_tasks': 8},
 {'user_id': 3, 'name': 'Clementine Bauch', 'completed_tasks': 7},
 {'user_id': 4, 'name': 'Patricia Lebsack', 'completed_tasks': 6},
 {'user_id': 5, 'name': 'Chelsey Dietrich', 'completed_tasks': 12},
 {'user_id': 6, 'name': 'Mrs. Dennis Schulist', 'completed_tasks': 6},
 {'user_id': 7, 'name': 'Kurtis Weissnat', 'completed_tasks': 9},
 {'user_id': 8, 'name': 'Nicholas Runolfsdottir V', 'completed_tasks': 11},
 {'user_id': 9, 'name': 'Glenna Reichert', 'completed_tasks': 8},
 {'user_id': 10, 'name': 'Clementina DuBuque', 'completed_tasks': 12}]